# Tabular Data RAG

In [14]:
import os
import pandas as pd

%pip install beautifultable
from beautifultable import BeautifulTable

Note: you may need to restart the kernel to use updated packages.


## Prepare Datasets

In [15]:
# years of evaluation
start_year = 2014
end_year = 2023

# datasets files
teams_dataset_file = os.path.join("datasets", "teams.csv")
standings_dataset_file = os.path.join("datasets" ,"standings.csv")
games_dataset_file = os.path.join("datasets", "games.csv")

teams_cleaned_dataset_file = os.path.join("datasets", "teams_clean.csv")
standings_cleaned_dataset_file = os.path.join("datasets" ,"standings_clean.csv")
games_cleaned_dataset_file = os.path.join("datasets", "games_clean.csv")

# seasons to evaluate
seasons_to_evaluate = [year for year in range(start_year, end_year + 1)]
# read teams dataset
teams = pd.read_csv(teams_dataset_file)

# removing irrelevant columns
teams.drop(["nfl", "nfl_team_id", "espn", "pfr", "pff", "pfflabel", "fo", "location", "short_location", "nickname", "hyphenated", "sbr", "sbr_wins", "sbr_name", "draft_kings"]
    , axis=1
    , inplace=True)
#display(teams.sample(10))
# keep last season records and set index on team short name
teams = teams[teams["season"] == end_year]
teams.set_index("team", inplace=True)
teams.drop("season", axis=1, inplace=True)
teams.rename(columns={"full" : "team_name"}, inplace=True)
#display(teams)
# dictionaries for relocated teams: {"PREV_TEAM_ID" : "NEW_TEAM_ID"}
relocated_teams = {"OAK" : "LV", "SD" : "LAC", "STL" : "LA"}

# generic function to replace relocated team ids
def replace_relocated_teams(target_data_frame, column_name, relocated_teams):
    for prev_team_id, new_team_id in relocated_teams.items():
        target_data_frame.loc[target_data_frame[column_name] == prev_team_id, column_name] = new_team_id